## Preparation

In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

2.4.0
sys.version_info(major=3, minor=7, micro=9, releaselevel='final', serial=0)
matplotlib 3.3.3
numpy 1.19.2
pandas 1.2.0
sklearn 0.24.0
tensorflow 2.4.0
tensorflow.keras 2.4.0


In [18]:
train_dir = "./datasets/10-monkey-species/training/training"
valid_dir = "./datasets/10-monkey-species/validation/validation"
label_file = "./datasets/10-monkey-species/monkey_labels.txt"

# Need to see the file structure!
print(os.listdir(train_dir))
print(os.listdir(valid_dir))

['n0', 'n1', 'n2', 'n3', 'n4', 'n5', 'n6', 'n7', 'n8', 'n9']
['n0', 'n1', 'n2', 'n3', 'n4', 'n5', 'n6', 'n7', 'n8', 'n9']


In [19]:
labels = pd.read_csv(label_file, header=0)
print(labels)

   Label     Latin Name              Common Name                     \
0  n0         alouatta_palliata\t    mantled_howler                   
1  n1        erythrocebus_patas\t    patas_monkey                     
2  n2        cacajao_calvus\t        bald_uakari                      
3  n3        macaca_fuscata\t        japanese_macaque                 
4  n4       cebuella_pygmea\t        pygmy_marmoset                   
5  n5       cebus_capucinus\t        white_headed_capuchin            
6  n6       mico_argentatus\t        silvery_marmoset                 
7  n7      saimiri_sciureus\t        common_squirrel_monkey           
8  n8       aotus_nigriceps\t        black_headed_night_monkey        
9  n9       trachypithecus_johnii    nilgiri_langur                   

    Train Images    Validation Images  
0             131                  26  
1             139                  28  
2             137                  27  
3             152                  30  
4             131 

In [20]:
# Image Scaling
height = 224
width = 224
channels = 3
batch_size = 24
num_classes = 10

train_datagen = keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function = keras.applications.resnet50.preprocess_input,
    # rescale = 1./255,  resnet50 has already done normalization!
    rotation_range = 40,
    width_shift_range = 0.2,   # percentage
    height_shift_range = 0.2,  # percentage
    shear_range = 0.2,         # shear strength
    zoom_range = 0.2,          # zoom strength
    horizontal_flip = True,
    fill_mode = "nearest"
)
train_generator = train_datagen.flow_from_directory(
    train_dir,                      # folder path
    target_size = (height, width),  # zoom size
    batch_size = batch_size,        # how many in one batch
    seed = 7,
    shuffle = True,
    class_mode = "categorical"      # before one_hot encode
)

Found 1098 images belonging to 10 classes.


In [21]:
valid_datagen = keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function = keras.applications.resnet50.preprocess_input
)
valid_generator = valid_datagen.flow_from_directory(
    valid_dir,                      # folder path
    target_size = (height, width),  # zoom size
    batch_size = batch_size,        # how many in one batch
    seed = 7,
    shuffle = False,
    class_mode = "categorical"      # before one_hot encode
)

Found 272 images belonging to 10 classes.


In [22]:
one = train_generator.next()[0]
one.shape

(24, 224, 224, 3)

In [25]:
one[0].shape

(224, 224, 3)

In [23]:
train_num = train_generator.samples  # 1098
valid_num = valid_generator.samples  # 272

In [24]:
# Display a mini-batch of size 24
for i in range(1):
    x, y = train_generator.next()
    print(x.shape, y.shape)
    print(y)

(24, 224, 224, 3) (24, 10)
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]]


## Modeling with ResNet-50

In [ ]:
resnet50_fine_tune = keras.models.Sequential()
resnet50_fine_tune.add(keras.applications.ResNet50(
    include_top=False,
    pooling='avg',
    weights='imagenet',
))
resnet50_fine_tune.add(keras.layers.Dense(num_classes, activation='softmax'))
resnet50_fine_tune.layers[0].trainable = False  # set ResNet (regarded as one layer)

resnet50_fine_tune.compile(loss="categorical_crossentropy",
                           optimizer="sgd",
                           metrics=["accuracy"])
resnet50_fine_tune.summary()

## Training

In [ ]:
valid_generator

In [ ]:
epochs = 10  # 10 for fine tune!
# fit fit_generator?
history = resnet50_fine_tune.fit_generator(
    train_generator, 
    steps_per_epoch = train_num // batch_size,
    epochs = epochs,
    validation_data = valid_generator,
    validation_steps = valid_num // batch_size
)

In [ ]:
def plot_learning_curves(history, label, epochs, min_value, max_value):
    data = {}
    data[label] = history.history[label]
    data['val_'+label] = history.history['val_'+label]
    pd.DataFrame(data).plot(figsize=(8, 5))
    plt.grid(True)
    plt.axis([0, epochs, min_value, max_value])
    plt.show()
    
plot_learning_curves(history, 'accuracy', epochs, 0, 1)
plot_learning_curves(history, 'loss', epochs, 0, 2)

## ResNet-50 with last a few layers trainable

In [ ]:
resnet50 = keras.applications.ResNet50(include_top=False,
                                       pooling="avg",
                                       weights="imagenet")
resnet50.summary()

In [ ]:
for layer in resnet50.layers[0:-5]:
    layer.trainable = False
    
resnet50_new = keras.models.Sequential([
    resnet50, keras.layers.Dense(num_classes, activation='softmax'),
])
resnet50_new.compile(loss="categorical_crossentropy", optimizer="sgd", metrics=['accuracy'])
resnet50_new.summary()

In [ ]:
epochs = 10  # 10 for fine tune!
history = resnet_new.fit_generator(
    train_generator, 
    steps_per_epoch = train_num // batch_size,
    epochs = epochs,
    validation_data = valid_generator,
    validation_steps = valid_num // batch_size
)